In [1]:
import json
import os
import numpy as np
from pprint import pprint
from copy import deepcopy

import GraphPlotter
import Helper
from ReadParameterFile import get_parameter_dict

In [2]:
params = get_parameter_dict('semiannually-params-json')
params

{'end_month': '12',
 'end_year': '2017',
 'graph_types': ['positive', 'negative', 'overall'],
 'input_dir': '../analysis/results',
 'languages': ['english', 'chinese', 'mix'],
 'leader_different': ['lim kit siang',
  'lim guan eng',
  'anwar',
  'mahathir',
  'abdul hadi awang',
  'liow tiong lai',
  'subramaniam',
  'najib'],
 'leader_same_bn': ['liow tiong lai',
  'subramaniam',
  'najib',
  'mah siew keong'],
 'leader_same_pakatan': ['lim kit siang',
  'lim guan eng',
  'anwar',
  'mahathir',
  'abdul hadi awang'],
 'output_dir': './results-json',
 'party_different': ['dap', 'pkr', 'ppbm', 'pas', 'mca', 'umno', 'mic'],
 'party_same_bn': ['mca', 'mic', 'umno', 'pbb', 'gerakan'],
 'party_same_pakatan': ['dap', 'pkr', 'pas', 'ppbm', 'amanah'],
 'start_month': '7',
 'start_year': '2017'}

In [3]:
def prep_data(params):
    data = json.load(open(os.path.join(params["input_dir"], "english_extracted.json")))
    data = data["data"]
    
    for k in data:
        raw_data = data[k]
    
        # Get length
        length = len(data[k][list(data[k].keys())[0]])

        overall = np.zeros(length, dtype=np.int32)
        for r_k in raw_data:
            raw_data[r_k] = np.array(raw_data[r_k])
            overall += raw_data[r_k]
        data[k] = raw_data
        data[k]["overall"] = overall
    
        """
        for _k in data[k]:
            data[k][_k] = Helper.summarize_values_to_week(data[k][_k])
        """
    return data

In [4]:
data = prep_data(params)

In [12]:
def filter_map(data, filter_list):
    ret_map = {}
    for name in filter_list:
        if name in data:
            ret_map[name] = data[name]
    
    return ret_map

In [6]:
def plot_helper(data, params, lang):
    date = params["start_year"] + "-" + params["start_month"] + " -- " + params["end_year"] + "-" + params["end_month"]
    
    cat_type = 'leader_different'
    filtered_dict = filter_map(data, params[cat_type])
    GraphPlotter.plot_semiannual_graph("Different coalition leader "+ date, filtered_dict, "leader_different", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition leader "+ date, filtered_dict, "leader_different", plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition leader "+ date, filtered_dict, "leader_different", plot_kind="negative", save=True, show=False, suffix=lang, params=params)

    cat_type = "party_different"
    filtered_dict = filter_map(data, params[cat_type])
    GraphPlotter.plot_semiannual_graph("Different coalition party", filtered_dict, "party_different", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition party", filtered_dict, "party_different", plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition party", filtered_dict, "party_different", plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "party_same_bn"
    filtered_dict = filter_map(data, params[cat_type])
    GraphPlotter.plot_semiannual_graph("BN party coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN party coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN party coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "party_same_pakatan"
    filtered_dict = filter_map(data, params[cat_type])
    GraphPlotter.plot_semiannual_graph("Pakatan party coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan party coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan party coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "leader_same_bn"
    filtered_dict = filter_map(data, params[cat_type])
    GraphPlotter.plot_semiannual_graph("BN leaders coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN leaders coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN leaders coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)

    cat_type = "leader_same_pakatan"
    filtered_dict = filter_map(data, params[cat_type])
    GraphPlotter.plot_semiannual_graph("Pakatan leaders coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan leaders coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan leaders coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)

In [9]:
for k in data:
    for _k in data[k]:
        data[k][_k] = Helper.summarize_values_to_week(data[k][_k])



dict_keys(['dap', 'pakatan', 'pakatan harapan', 'pas', 'pakatan rakyat', 'gerakan', 'mahathir', 'mca', 'najib', 'bn', 'amanah', 'pkr', 'lks', 'rosmah', 'ph', 'tony pua', 'rafizi', 'anwar', 'mukhriz', 'hishammuddin', 'wan azizah', 'lim kit siang', 'ppbm', 'tengku razaleigh', 'muhyiddin', 'khairy', 'lim guan eng', 'supp', 'parti pribumi bersatu malaysia', 'pbb', 'azmin ali', 'ldp', 'pbs', 'nurul izzah', 'karpal singh', 'liow tiong lai', 'shahrizat', 'zairil khir johari', 'ahmad yakob'])
{'negative': [0,
              1,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0],
 'neutral': [1,
             1,
             3,
  

/usr/lib/python3.6/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The set_color_cycle attribute was deprecated in version 1.5. Use set_prop_cycle instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


leader_different_overall_english.png
leader_different_positive_english.png
leader_different_negative_english.png
dict_keys(['dap', 'pakatan', 'pakatan harapan', 'pas', 'pakatan rakyat', 'gerakan', 'mahathir', 'mca', 'najib', 'bn', 'amanah', 'pkr', 'lks', 'rosmah', 'ph', 'tony pua', 'rafizi', 'anwar', 'mukhriz', 'hishammuddin', 'wan azizah', 'lim kit siang', 'ppbm', 'tengku razaleigh', 'muhyiddin', 'khairy', 'lim guan eng', 'supp', 'parti pribumi bersatu malaysia', 'pbb', 'azmin ali', 'ldp', 'pbs', 'nurul izzah', 'karpal singh', 'liow tiong lai', 'shahrizat', 'zairil khir johari', 'ahmad yakob'])
{'negative': [6,
              5,
              9,
              5,
              1,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
      

              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0],
 'neutral': [0,
             1,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0],
 'overall': [0,
             4,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
             0,
       

party_same_pakatan_negative_english.png
dict_keys(['dap', 'pakatan', 'pakatan harapan', 'pas', 'pakatan rakyat', 'gerakan', 'mahathir', 'mca', 'najib', 'bn', 'amanah', 'pkr', 'lks', 'rosmah', 'ph', 'tony pua', 'rafizi', 'anwar', 'mukhriz', 'hishammuddin', 'wan azizah', 'lim kit siang', 'ppbm', 'tengku razaleigh', 'muhyiddin', 'khairy', 'lim guan eng', 'supp', 'parti pribumi bersatu malaysia', 'pbb', 'azmin ali', 'ldp', 'pbs', 'nurul izzah', 'karpal singh', 'liow tiong lai', 'shahrizat', 'zairil khir johari', 'ahmad yakob'])
{'negative': [0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0],
 'neutral'

In [13]:
plot_helper(data, params, "english")

/usr/lib/python3.6/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The set_color_cycle attribute was deprecated in version 1.5. Use set_prop_cycle instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


leader_different_overall_english.png
leader_different_positive_english.png
leader_different_negative_english.png
party_different_overall_english.png
party_different_positive_english.png
party_different_negative_english.png
party_same_bn_overall_english.png
party_same_bn_positive_english.png
party_same_bn_negative_english.png
party_same_pakatan_overall_english.png
party_same_pakatan_positive_english.png
party_same_pakatan_negative_english.png
leader_same_bn_overall_english.png
leader_same_bn_positive_english.png
leader_same_bn_negative_english.png
leader_same_pakatan_overall_english.png
leader_same_pakatan_positive_english.png
leader_same_pakatan_negative_english.png
